In [2]:
import pandas as pd
import re

In [3]:
#cleaning raw recipe data, dropping unneccesary columns, splitting nutrition info and converting it to numeric values
df_recipes = pd.read_csv("RAW_recipes.csv")

#eliminate desserts
df_recipes = df_recipes[df_recipes["tags"].str.contains('dessert') == False]
#drop unneeded columns
df_recipes = df_recipes.drop(columns = ["id", "minutes", "contributor_id", "submitted", "tags", "n_steps", "description"])
df_recipes = df_recipes.drop(columns = "steps")
#split, clean, numerify nutritional information
df_recipes[['calories (number)', 'total fat (PDV)', 'sugar (PDV)' , 'sodium (PDV)' , 'protein (PDV)' , 'saturated fat (PDV)', 'carbohydrates (PDV)']]=df_recipes['nutrition'].str.split(', ', expand=True,n=6)
df_recipes['calories (number)'] = df_recipes['calories (number)'].str.lstrip('[')
df_recipes = df_recipes.drop(columns = ["saturated fat (PDV)", "carbohydrates (PDV)", 'nutrition'])
df_recipes[['calories (number)', 'total fat (PDV)', 'sugar (PDV)' , 'sodium (PDV)' , 'protein (PDV)']] = df_recipes[['calories (number)', 'total fat (PDV)', 'sugar (PDV)' , 'sodium (PDV)' , 'protein (PDV)']].astype(float)
df_recipes

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
0,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0
1,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0
4,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0
...,...,...,...,...,...,...,...,...
231630,zydeco sauce,"['mayonnaise', 'prepared horseradish', 'worces...",6,239.9,30.0,19.0,22.0,1.0
231631,zydeco shrimp wrap,"['white rice', 'vegetable oil', 'onion', 'gree...",11,482.6,13.0,25.0,37.0,33.0
231632,zydeco soup,"['celery', 'onion', 'green sweet pepper', 'gar...",22,415.2,26.0,34.0,26.0,44.0
231633,zydeco spice mix,"['paprika', 'salt', 'garlic powder', 'onion po...",13,14.8,0.0,2.0,58.0,1.0


In [4]:
#create df of beef dishes by keyword
search_values = ['beef', 'hamburger', 'steak', 'veal', 'bison', 'buffalo']
beef_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values))]
beef_df = pd.DataFrame(beef_df)
beef_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0
18,chinese chop suey,"['celery', 'onion', 'ground pork', 'soy sauce'...",7,395.4,31.0,20.0,29.0,51.0
26,fool the meat eaters chili,"['vegetarian ground beef', 'garlic', 'onion', ...",12,295.6,3.0,56.0,76.0,32.0
27,forgotten minestrone,"['beef stew meat', 'water', 'tomatoes', 'beef ...",14,346.9,24.0,18.0,42.0,42.0
31,global gourmet taco casserole,"['ground beef', 'onion', 'tomato sauce', 'taco...",12,456.8,40.0,34.0,67.0,57.0
...,...,...,...,...,...,...,...,...
231583,zuppa di cipolla al vino rosso,"['beef broth', 'butter', 'onions', 'flour', 'o...",10,9282.1,136.0,340.0,715.0,746.0
231604,zurcher geschnetzeltes zurich style veal stri...,"['veal cutlets', 'all-purpose flour', 'butter'...",14,478.8,51.0,8.0,7.0,63.0
231605,zurich diced veal zurcher geschnetzeltes,"['onion', 'mushrooms', 'veal', 'flour', 'butte...",10,562.5,58.0,11.0,10.0,62.0
231606,zurich ragout zrcher geschnetzeltes,"['veal', 'veal kidneys', 'clarified butter', '...",13,278.8,22.0,5.0,12.0,38.0


In [5]:
# a serving of beef contains ~ 200 calories, so we're discarding all entries with calorie values much lower or higher than a realistic serving
beef_df = beef_df[(beef_df["calories (number)"] <700) & (beef_df["calories (number)"] >200) & (beef_df['sugar (PDV)']<50)]
beef_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0
18,chinese chop suey,"['celery', 'onion', 'ground pork', 'soy sauce'...",7,395.4,31.0,20.0,29.0,51.0
27,forgotten minestrone,"['beef stew meat', 'water', 'tomatoes', 'beef ...",14,346.9,24.0,18.0,42.0,42.0
31,global gourmet taco casserole,"['ground beef', 'onion', 'tomato sauce', 'taco...",12,456.8,40.0,34.0,67.0,57.0
43,immoral sandwich filling loose meat,"['ground beef', 'sugar', 'prepared yellow must...",8,223.2,22.0,4.0,7.0,35.0
...,...,...,...,...,...,...,...,...
231547,zucchini salsa burgers,"['ground beef', 'breadcrumbs', 'egg', 'zucchin...",13,571.0,47.0,29.0,34.0,59.0
231604,zurcher geschnetzeltes zurich style veal stri...,"['veal cutlets', 'all-purpose flour', 'butter'...",14,478.8,51.0,8.0,7.0,63.0
231605,zurich diced veal zurcher geschnetzeltes,"['onion', 'mushrooms', 'veal', 'flour', 'butte...",10,562.5,58.0,11.0,10.0,62.0
231606,zurich ragout zrcher geschnetzeltes,"['veal', 'veal kidneys', 'clarified butter', '...",13,278.8,22.0,5.0,12.0,38.0


In [6]:
search_values2 = ['pork', 'bacon', 'pancetta', 'boar', 'sausage']
pork_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values2))]
pork_df = pd.DataFrame(pork_df)
pork_df = pork_df[(pork_df["calories (number)"] <700) & (pork_df["calories (number)"] >180)&(pork_df['sugar (PDV)']<50)]
pork_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
18,chinese chop suey,"['celery', 'onion', 'ground pork', 'soy sauce'...",7,395.4,31.0,20.0,29.0,51.0
51,killer lasagna,"['italian sausage', 'ground beef', 'garlic', '...",15,370.5,38.0,21.0,57.0,53.0
64,mr grant you took half veal prince orloff,"['veal roast', 'butter', 'oil', 'carrots', 'on...",11,510.6,50.0,10.0,21.0,90.0
99,the woiks dilly burgers,"[""mccormick's montreal brand steak seasoning"",...",9,449.2,50.0,16.0,19.0,58.0
125,2000 meatloaf,"['meatloaf mixture', 'unsmoked bacon', 'goat c...",13,267.0,30.0,12.0,12.0,29.0
...,...,...,...,...,...,...,...,...
231598,zuppa toscana from olive garden,"['mild italian sausage', 'crushed red pepper f...",10,531.7,59.0,13.0,74.0,38.0
231599,zuppa toscana soup,"['chicken stock', 'light cream', 'russet potat...",7,684.1,70.0,24.0,68.0,59.0
231600,zuppa toscana soup olive garden clone,"['chicken stock', 'bacon', 'onion white', 'rus...",14,432.8,32.0,29.0,39.0,42.0
231601,zuppa toscana soup light version,"['hot italian turkey sausage', 'water', 'chick...",8,278.3,10.0,36.0,30.0,34.0


In [7]:
search_values3 = ['chicken', 'turkey']
chix_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values3))]
chix_df = pd.DataFrame(chix_df)
chix_df = chix_df[(chix_df["calories (number)"] <700) & (chix_df["calories (number)"] >170)&(chix_df['sugar (PDV)']<50)]
chix_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
22,crispy crunchy chicken,"['boneless skinless chicken breast halves', 'c...",10,335.8,11.0,2.0,24.0,64.0
57,make that chicken dance salsa pasta,"['tomatoes', 'garlic', 'onion', 'button mushro...",15,255.2,18.0,39.0,18.0,39.0
73,open sesame noodles,"['vermicelli', 'rice vinegar', 'reduced sodium...",12,617.5,34.0,13.0,20.0,56.0
75,pick me up party chicken kabobs,"['boneless chicken breast', 'garlic', 'salt', ...",10,213.6,24.0,0.0,26.0,31.0
85,smells like sunday chicken fricassee with mea...,"['boneless skinless chicken thighs', 'all-purp...",15,692.2,53.0,16.0,17.0,102.0
...,...,...,...,...,...,...,...,...
231600,zuppa toscana soup olive garden clone,"['chicken stock', 'bacon', 'onion white', 'rus...",14,432.8,32.0,29.0,39.0,42.0
231601,zuppa toscana soup light version,"['hot italian turkey sausage', 'water', 'chick...",8,278.3,10.0,36.0,30.0,34.0
231607,zurichgeschnetzeltes or diced chicken,"['chicken breasts', 'mushrooms', 'lemon juice'...",7,592.2,55.0,6.0,7.0,110.0
231626,zwiebelsuppe clear german onion soup,"['onions', 'red onions', 'fresh garlic', 'oliv...",11,252.0,17.0,21.0,44.0,21.0


In [8]:
search_values4 = ['duck', 'pheasant', 'rabbit', 'quail', 'lapin', 'goose', 'grouse']
small_game_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values4))]
small_game_df = pd.DataFrame(small_game_df)
small_game_df = small_game_df[(small_game_df["calories (number)"] <700) & (small_game_df["calories (number)"] >170)&(small_game_df['sugar (PDV)']<50)]
#removed "duck sauce" from results because it was skewing data
small_game_df = small_game_df[small_game_df['ingredients'].str.contains("duck sauce")==False]
small_game_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
15786,balsamic duck breasts and asparagus pure,"['duck breasts', 'salt & pepper', 'balsamic vi...",8,671.9,45.0,29.0,9.0,128.0
19222,bbq duck and ramen soup,"['water', 'chicken stock', 'red chilies', 'gin...",15,282.8,14.0,21.0,32.0,21.0
21358,beer braised rabbit or chicken for the crock...,"['rabbit', 'olive oil', 'potatoes', 'carrots',...",14,598.9,30.0,39.0,27.0,101.0
21581,belgian hare a la maryland,"['rabbit', 'salt and pepper', 'flour', 'eggs',...",8,258.4,21.0,3.0,5.0,13.0
27701,braised rabbit,"['flour', 'salt', 'pepper', 'rabbit', 'egg', '...",8,412.3,34.0,7.0,33.0,16.0
...,...,...,...,...,...,...,...,...
217772,turducken roulade,"['boneless turkey breast', 'boneless duck brea...",6,249.6,22.0,0.0,6.0,55.0
220110,upland stir fry,"['buttermilk baking mix', 'pepper', 'pheasant'...",11,545.2,48.0,35.0,38.0,67.0
224452,warm oriental duck and mango salad,"['duck breasts', 'five-spice powder', 'sunflow...",13,314.3,22.0,46.0,19.0,59.0
224511,warm salad of duck with sesame and ginger,"['duck breasts', 'vegetable oil', 'sesame seed...",11,361.6,37.0,13.0,4.0,60.0


In [9]:
search_values5 = ['venison', 'deer', 'elk', 'caribou', 'moose']
big_game_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values5))]
big_game_df = pd.DataFrame(big_game_df)
big_game_df = big_game_df[(big_game_df["calories (number)"] <700) & (big_game_df["calories (number)"] >170)&(big_game_df['sugar (PDV)']<50)]
big_game_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
33,grilled venison burgers,"['ground venison', 'egg substitute', 'non-fat ...",10,190.9,10.0,10.0,10.0,45.0
2763,african piri piri kabobs,"['sweet potato', 'venison', 'red sweet peppers...",13,317.6,15.0,30.0,16.0,56.0
4896,amethyst s paleo elk meatloaf,"['ground elk', 'egg', 'carrot', 'celery', 'oni...",10,290.6,22.0,34.0,19.0,54.0
12768,bacon wrapped venison tenderloin with portabel...,"['venison tenderloin', 'bacon', 'yellow onion'...",9,323.7,47.0,7.0,32.0,15.0
15490,baked venison beef parmesan,"['venison steak', 'fresh breadcrumb', 'parmesa...",10,458.5,30.0,30.0,35.0,80.0
...,...,...,...,...,...,...,...,...
222825,venison tenderloin with cabernet sauce,"['venison loin', 'garlic', 'salt and black pep...",11,188.2,12.0,1.0,5.0,26.0
222827,venison tips and noodles,"['venison stew meat', 'condensed cream of mush...",8,513.6,15.0,14.0,17.0,68.0
222829,venison with plums and calvados,"['venison tenderloins', 'olive oil', 'plums', ...",10,245.3,15.0,39.0,0.0,50.0
223209,very versatile venison stew,"['ground venison', 'onions', 'potatoes', 'fres...",8,172.3,6.0,20.0,13.0,29.0


In [10]:
search_values6 = ['fish', 
                  'shrimp',
                  'crab',
                  'salmon',
                  'tuna',
                  'swordfish',
                  'mackerel',
                  'bass',
                  'perch',
                  'oyster',
                  'scallop',
                  'lobster',
                  'prawn',
                  'squid',
                  'octopus',
                  'monkfish',
                  'black cod',
                  'trout',
                  'mahi-mahi', 
                  'pollock']

seafood_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(search_values6))]
seafood_df = pd.DataFrame(seafood_df)
seafood_df = seafood_df[(seafood_df["calories (number)"] <700) & (seafood_df["calories (number)"] >150)&(seafood_df['sugar (PDV)']<50)]
seafood_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
78,put down your fork tuna and bean salad,"['light olive oil', 'red wine vinegar', 'garli...",11,227.2,14.0,8.0,29.0,40.0
105,voodoo calamari ink pasta,"['squid', 'spaghetti', 'dry white wine', 'toma...",9,458.1,24.0,6.0,4.0,50.0
113,1 favorite chinese steamed whole fish by sy,"['fish', 'scallions', 'rice wine', 'peanut oil...",9,264.7,41.0,4.0,36.0,3.0
159,big easy gumbo,"['vegetable oil', 'all-purpose flour', 'onion'...",12,484.1,42.0,7.0,32.0,75.0
163,california roll salad,"['white rice', 'seasoned rice vinegar', 'lettu...",8,455.7,13.0,38.0,72.0,34.0
...,...,...,...,...,...,...,...,...
231585,zuppa di pesce,"['squid', 'large shrimp', 'black pepper', 'sal...",21,562.8,31.0,25.0,53.0,99.0
231586,zuppa di pesce castagna,"['leeks', 'fennel bulb', 'olive oil', 'arbol c...",20,392.5,29.0,11.0,43.0,68.0
231587,zuppa di pesce cioppino or fish stew,"['olive oil', 'onions', 'garlic cloves', 'anch...",29,160.3,3.0,17.0,31.0,37.0
231619,zwicker inn seafood chowder,"['haddock', 'lobster', 'scallops', 'butter', '...",14,482.7,58.0,19.0,9.0,28.0


In [11]:
all_search = (search_values+
              search_values2+
              search_values3+
              search_values4+ 
            search_values5+ 
            search_values6)

veg_other_df = df_recipes[df_recipes["ingredients"].str.contains('|'.join(all_search))==False]
veg_other_df = pd.DataFrame(veg_other_df)
veg_other_df = veg_other_df[(veg_other_df["calories (number)"] <700) & (veg_other_df["calories (number)"] >150)&(veg_other_df['sugar (PDV)']<50)]
veg_other_df

,name,ingredients,n_ingredients,calories (number),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV)
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0
6,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...",9,380.7,53.0,7.0,24.0,6.0
19,cream of cauliflower soup vegan,"['canola oil', 'onion', 'garlic', 'cauliflower...",16,174.2,4.0,24.0,1.0,15.0
25,emotional balance spice mixture,"['ground black pepper', 'ground ginger', 'grou...",6,182.7,11.0,4.0,1.0,13.0
36,how i got my family to eat spinach spinach ca...,"['frozen chopped spinach', 'egg', 'salt', 'bla...",8,166.1,16.0,6.0,32.0,19.0
...,...,...,...,...,...,...,...,...
231580,zuppa de clams or mussels,"['baby clams', 'salt and pepper', 'olive oil',...",8,358.1,56.0,21.0,0.0,3.0
231589,zuppa di porcini porcini mushrooms soup,"['porcini mushrooms', 'extra virgin olive oil'...",8,207.0,33.0,0.0,1.0,6.0
231603,zupse bread swiss bread,"['flour', 'butter', 'milk', 'salt', 'sugar', '...",7,467.4,18.0,2.0,42.0,26.0
231622,zwiebelbrotchen onion rolls,"['onions', 'sesame oil', 'yeast', 'water', 'su...",9,157.9,3.0,7.0,0.0,8.0


In [12]:
beef_df.to_csv("df_beef.csv")
pork_df.to_csv("df_pork.csv")
chix_df.to_csv("df_chix.csv")
small_game_df.to_csv("df_small_game.csv")
big_game_df.to_csv("df_big_game.csv")
seafood_df.to_csv("df_seafood.csv")
veg_other_df.to_csv("df_veg_other.csv")